In [1]:
import pandas as pd
import xlwings as xw
from xlwings import constants

In [2]:
data = pd.DataFrame([[1,1,1,1],[2,2,2,2]],columns = ['Col1','Col2','Col3','Col4'])
data

,Col1,Col2,Col3,Col4
0,1,1,1,1
1,2,2,2,2


In [3]:
wb = xw.Book()
sht = wb.sheets[0]
sht.name = 'Raw Data'
sht.range('A1').options(index=False).value = data
sht.cells.api.Font.Size = 8

xldata = sht.range('A:D')

xldata.number_format = '#,##0;[red]-#,##0;-'
xldata.autofit()

sht2 = wb.sheets.add('Pivot')

In [4]:
PivotCache = wb.api.PivotCaches().Create(SourceType=constants.PivotTableSourceType.xlDatabase,
                                         SourceData=xldata.api, 
                                         Version=constants.PivotTableVersionList.xlPivotTableVersion14)

PivotTable = PivotCache.CreatePivotTable(TableDestination=sht2.range('A1').api,
                                         TableName='PivotTable1',
                                         DefaultVersion=constants.PivotTableVersionList.xlPivotTableVersion14)
                                         
#change to old style                                         
PivotTable.InGridDropZones = True
PivotTable.RowAxisLayout(constants.LayoutRowType.xlTabularRow)

#stop column autoformat
PivotTable.HasAutoFormat = False
#turn off row totals
PivotTable.RowGrand = False

In [5]:
#place the data
PivotTable.PivotFields('Col1').Orientation = constants.PivotFieldOrientation.xlRowField
PivotTable.PivotFields('Col1').Position = 1
PivotTable.PivotFields('Col2').Orientation = constants.PivotFieldOrientation.xlRowField
PivotTable.PivotFields('Col2').Position = 2
PivotTable.PivotFields('Col3').Orientation = constants.PivotFieldOrientation.xlColumnField
PivotTable.PivotFields('Col3').Position = 1
PivotTable.PivotFields("Col4").Orientation = constants.PivotFieldOrientation.xlDataField

In [6]:
#get rid of blanks
PivotTable.PivotFields('Col3').PivotItems('(blank)').Visible = False
#format 
PivotTable.PivotFields('Sum of Col4').NumberFormat = '#,##0;[red]-#,##0;-'
#get rd of subtotals
PivotTable.PivotFields("Col1").Subtotals = [False]*12

#sheet format
sht2.cells.api.Font.Size = 8
sht2.range('A:H').autofit()
sht2.range("C:D").api.HorizontalAlignment = constants.HAlign.xlHAlignCenter